# Tyler's Workspace

In [88]:
# imports we need for the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.dummy import DummyClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [2]:
# change display to show all columns of the dataframe
pd.set_option('display.max_columns', None)

In [3]:
# here we have the features provided to us
X = pd.read_csv('data/TrainingFeatures.csv', parse_dates=[2])
X

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kiduruni,Kilimanjaro,3,5,Hai,Masama Magharibi,125,True,GeoData Consultants Ltd,Water Board,Losaa Kia water supply,True,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Igumbilo,Iringa,11,4,Njombe,Ikondo,56,True,GeoData Consultants Ltd,VWC,Ikondo electrical water sch,True,1996,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,Rufiji,Madungulu,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,Mwinyi,Dodoma,1,4,Chamwino,Mvumi Makulu,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


In [4]:
# and the target values with matching index and id
y = pd.read_csv('data/TrainingLabels.csv')
y

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
...,...,...
59395,60739,functional
59396,27263,functional
59397,37057,functional
59398,31282,functional


## Data Exploration

In [5]:
# the data is mostly clean, but some variables do have several missing values, let's look into them
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     59400 non-null  int64         
 1   amount_tsh             59400 non-null  float64       
 2   date_recorded          59400 non-null  datetime64[ns]
 3   funder                 55765 non-null  object        
 4   gps_height             59400 non-null  int64         
 5   installer              55745 non-null  object        
 6   longitude              59400 non-null  float64       
 7   latitude               59400 non-null  float64       
 8   wpt_name               59400 non-null  object        
 9   num_private            59400 non-null  int64         
 10  basin                  59400 non-null  object        
 11  subvillage             59029 non-null  object        
 12  region                 59400 non-null  object        
 13  r

In [6]:
# looks like there are lots of one-off and unknown funders, it may be worth it to only
# include the funders who show up often, and bin all the others in an other category
X['funder'].value_counts()

Government Of Tanzania           9084
Danida                           3114
Hesawa                           2202
Rwssp                            1374
World Bank                       1349
                                 ... 
Tlc/community                       1
Liz                                 1
Ju-sarang Church' And Bugango       1
People Of Sweden                    1
Agape Churc                         1
Name: funder, Length: 1897, dtype: int64

### Missing Values

In [7]:
# it would probably be wise to do a similiar thing to installer as funder
X['installer'].value_counts()

DWE                      17402
Government                1825
RWE                       1206
Commu                     1060
DANIDA                    1050
                         ...  
Hamisi Fidia                 1
CHONJA CHARLES               1
Masjid Nnre                  1
Upendo primary School        1
BGSS                         1
Name: installer, Length: 2145, dtype: int64

In [8]:
# hmmm, should we impute these as True, seems intuitive to,
# but we'll need to investigate it more 
X['public_meeting'].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

In [9]:
# should we impute the nulls as either other or None
X['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [10]:
# definitely have way to many different scheme names, over 2000,
# we'll need to trucate this list somehow to use it
# and I need to better understand what it means
# may be best just to not use this feature
X['scheme_name'].value_counts()

K                              682
None                           644
Borehole                       546
Chalinze wate                  405
M                              400
                              ... 
Mradi wa maji wa Ng`ambo         1
Kashishi water supply            1
Nywang'hwale                     1
Sasaka village water supply      1
Mkuini paralle line              1
Name: scheme_name, Length: 2696, dtype: int64

In [11]:
# We could create a new Missing value, or impute it as either True or False
# need to look deeper into this
X['permit'].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

### Summary Stats

In [12]:
# looking at the summary statistics of numeric variables
X.describe(datetime_is_numeric=True)

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,2012-03-29 09:11:33.818181888,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
min,0.000000,0.000000,2002-10-14 00:00:00,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,2011-04-01 00:00:00,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,2012-10-10 00:00:00,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,2013-02-09 00:00:00,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2013-12-03 00:00:00,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000
std,21453.128371,2997.574558,NaN,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547


In [13]:
# looking at how many region codes there are
len(X['region_code'].unique())

27

In [14]:
len(X['district_code'].unique())

20

#### Numeric variables
- id: identifying variable unique to every waterwell, descriptive statistics are not important
- amount_tsh: Total static head is the vertical distance between the water source and the head of the pump
    - distance is zero for the vast majority of wells
- date_recorded: The day the data from the well was recorded
    - seems the vast majority of the dates were in the early 2010's
- gps_height: The altitude of the well
    - seems that most wells were quite low in elevation, which intuitively makes sense
- longitude: The longitudinal coordinate of the well
- latitude: The lattitudinal coordinate of the well
- num_private: Could not find what num_private means anywhere
    - good candidate feature to be dropped
- region_code: Contains some sort of geographic data
    - there are 21 different regions, but 27 different region_codes
- population: Described as population around the well
    - not exactly sure what this means
    - regardless, most wells have very low numbers
    - all those zeroes could be missing values or reflective of what "around the well" means
- construction_year: The year that the well was built
    - all the zeroes are definitely missing values here

In [15]:
# looked at all the categorical variables
X.describe(include='object')

,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625


## Data Cleaning

##### Scheme Name

In [16]:
# scheme_name is super messy, it may be best to drop it
X['scheme_name'].value_counts().head(20)

K                                        682
None                                     644
Borehole                                 546
Chalinze wate                            405
M                                        400
DANIDA                                   379
Government                               320
Ngana water supplied scheme              270
wanging'ombe water supply s              261
wanging'ombe supply scheme               234
Bagamoyo wate                            229
I                                        229
Uroki-Bomang'ombe water sup              209
N                                        204
Kirua kahe gravity water supply trust    193
Machumba estate pipe line                185
Makwale water supplied sche              166
Kijiji                                   161
S                                        154
mtwango water supply scheme              152
Name: scheme_name, dtype: int64

##### Payment

In [17]:
# the payment and payment_type features are identical
X['payment'].value_counts()

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [18]:
X['payment_type'].value_counts()

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

##### Water Quality

In [19]:
# The water_quality and quality_group features have a ton of overlap
X['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [20]:
X['quality_group'].value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

##### Water Quantity

In [21]:
# No further information provided between quantity and quantity_group
X['quantity'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [22]:
X['quantity_group'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

##### Source

In [23]:
# tons of redundancy between source and source_type
X['source'].value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [24]:
X['source_type'].value_counts()

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

##### Waterpoint Type

In [25]:
# Only one extra category for waterpoint_type, probably good to drop waterpoint_type_group
X['waterpoint_type'].value_counts()

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

In [26]:
X['waterpoint_type_group'].value_counts()

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

##### Extraction Type

In [27]:
# extraction_type, extraction_type_group, and extraction_type_class 
# all give the same information with different levels of granularity
X['extraction_type'].value_counts()

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64

In [28]:
X['extraction_type_group'].value_counts()

gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: extraction_type_group, dtype: int64

In [29]:
X['extraction_type_class'].value_counts()

gravity         26780
handpump        16456
other            6430
submersible      6179
motorpump        2987
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

##### Region

In [30]:
# There is conflicting information between region and region_code
X.groupby('region')['region_code'].value_counts()

region         region_code
Arusha         2              3024
               24              326
Dar es Salaam  7               805
Dodoma         1              2201
Iringa         11             5294
Kagera         18             3316
Kigoma         16             2816
Kilimanjaro    3              4379
Lindi          80             1238
               8               300
               18                8
Manyara        21             1583
Mara           20             1969
Mbeya          12             4639
Morogoro       5              4006
Mtwara         90              917
               99              423
               9               390
Mwanza         19             3047
               17               55
Pwani          6              1609
               60             1025
               40                1
Rukwa          15             1808
Ruvuma         10             2640
Shinyanga      17             4956
               14               20
               11           

##### Management

In [31]:
# scheme_management seems to describe something different from the other two management variables,
# but also seems to have high levels of multicollinearity. However, management seems to be a more granular 
# version of the management group variable.
X['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [32]:
X['management'].value_counts()

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [33]:
X['management_group'].value_counts()

user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

##### Subvillage

In [34]:
# many of the subvillages with few observations won't be conducive to modeling
(X['subvillage'].value_counts() > 100).sum()

22

In [35]:
# it may be best just to drop the column given how the vast majority of subvillages will only have 1 well
X['subvillage'].value_counts().head(22)

Madukani      508
Shuleni       506
Majengo       502
Kati          373
Mtakuja       262
Sokoni        232
M             187
Muungano      172
Mbuyuni       164
Mlimani       152
Songambele    147
Msikitini     134
Miembeni      134
1             132
Kibaoni       114
Kanisani      111
Mapinduzi     109
I             109
Mjimwema      108
Mjini         108
Mkwajuni      104
Mwenge        102
Name: subvillage, dtype: int64

##### District Code

In [36]:
# district code seems really messy, it would have to be grouped by region
# may be better not to use this feature
X.groupby('region')['district_code'].value_counts()

region  district_code
Arusha  2                1206
        7                1009
        30                326
        6                 310
        5                 201
                         ... 
Tanga   5                 293
        8                 288
        6                 266
        7                 127
        4                 110
Name: district_code, Length: 132, dtype: int64

#### Categorical variables
- funder: The name of the party that funded the well
- installer: The name of the party that installed the well
- wpt_name: The name of the waterpoint
    - a good candidate for dropping
- basin: The name of the basin that the well's water drains to
- subvillage: Contains some sort of geographic data
    - extremely granular data, almost 20,000 unique values
    - will need to either drop or transform data
- region: Contains some sort of geographic data
- lga: Contains some sort of geographic data
    - local government authority???
- ward: Contains some sort of geographic data
    - probably a better candidate for analysis than subvillage
- public_meeting: not sure what this point means
    - presumably just true if the well is used as a place of public meeting
- recorded_by: The party that recorded the data
    - literally only one category, should drop this feature
- scheme_management: Describes who operates the waterpoint
    - seems to classify the operators of the well by type of party
- scheme_name: Describes who operates the waterpoint
    - seems to give the name of the party
    - way more observations here, may want to
- permit: Is true if the waterpoint has a permit
- extraction_type: Describes the method of extraction used
    - 17 unique categories, but several don't have many observations
- extraction_type_group: Describes the method of extraction used
    - 13 unique categories, and each one has at least 98 observations
- extraction_type_class: Describes the method of extraction used
    - 7 unique categories, each one has enough observations but group has more granularity
- management: Describes the type of party that manages the well
    - 12 unique values, look at scheme_management
- management_group: Describes the type of party that manages the well
    - 5 uniqe values
- payment: Describes the cost of operating the well
    - No idea if cost for the users or for the operating party
- payment_group: Describes the cost of operating the well
    - Literally all the same values as payment, drop this column
- water_quality: Describes the quality of the water
    - may be worth combining salty abandoned and flouride abandoned into just abandoned
- quality_group: Describes the quality of the water
    - has significant overlap with water_quality, may just drop this feature
- quantity: Describes the quantity of water the well can provide
    - this feature almost seems like cheating, may have misunderstood the problem
- quantity_group: Describes the quantity of water the well can provide
    - once again a redundant feature with no new info, need to drop this feature
- source: Describes where the water comes from
- source_type: Describes where the water comes from
    - lots of redundancy between this feature and source
- source_class: Describes where the water comes from
- waterpoint_type: Describes the type of waterpoint
- waterpoint_type_group: Describes the type of waterpoint
    - need to drop either waterpoint type or waterpoint_type_group

In [37]:
# just to look at the target
y.describe(include='object')

,status_group
count,59400
unique,3
top,functional
freq,32259


In [38]:
# and looked at the split up of the target, may want to make it binary
y['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [39]:
# dropped the id column from X and y, shouldn't need them,
# and can join them back later if necessary or just rerun notebook
#X = X.drop('id', axis=1)
#y = y.drop('id', axis=1)

### Pre-split Cleaning

#### Feature Removal

First drop all the columns that are too messy or don't provide any further information. The `num_private` feature has 0's for the vast majority of the columns and there is record of what this even means. The `recorded_by` feature is the same for every single well, and thus provides no meaningful information to our model. The `payment_type` variable is identical to the `payment` variable. The `quality_group` contains almost identical information to `water_quality`. The `quantity_group` feature contains identical information to the `quantity` variable. The `waterpoint_type_group` feature has the exact same information as `waterpoint_type` except the latter splits up communal standpipe category into two types. The `wpt_name` feature only contains identifying information, specifically the name of the well. The `date_recorded` variable shouldn't be used as a predictor as this is not a time series prediction. The `scheme_name` variable seems too messy to effictively use, with far too many NA's and uninterpretable categories. The `extraction_type`, `extraction_type_group`, and `extraction_type_class` variables all have overlapping information, but only the `extraction_type_group` feature seemed to hit the perfect balance of category size. The `region_code` variable is a different form of the `region` variable, but the two features have a lot of conflicting information. The `region` feature writes out each region name and seems less likely to run into data entry errors, so `region_code` will be dropped. The `source_type` variable has the same information as `source`, but with less granularity. The vast majority of `subvillage` categories only have one well, it seems unlikely this feature would have much useful information and would add at least 20 more columns of information once encoded.

In [40]:
# The models shouldn't suffer from dropping these columns
drop_cols = ['num_private', 'recorded_by', 'payment_type', 'quality_group','wpt_name', 'management_group',
             'quantity_group','waterpoint_type_group', 'date_recorded', 'scheme_name', 'district_code',
             'extraction_type', 'extraction_type_class', 'region_code', 'source_type', 'subvillage']
X.drop(drop_cols, axis=1, inplace=True)

In [41]:
X[X['amount_tsh'] > 100][X['extraction_type_group'] == 'swn 80']

<ipython-input-41-5bce87fe59d0>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X[X['amount_tsh'] > 100][X['extraction_type_group'] == 'swn 80']


,id,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type_group,management,payment,water_quality,quantity,source,source_class,waterpoint_type
11,50409,200.0,Danida,1062,DANIDA,35.770258,-10.574175,Lake Nyasa,Ruvuma,Namtumbo,Msindo,250,True,WUG,False,1987,swn 80,wug,pay when scheme fails,soft,insufficient,shallow well,groundwater,hand pump
70,21990,200.0,Government Of Tanzania,303,RWE,35.818981,-8.934950,Rufiji,Morogoro,Kilombero,Utengule,280,True,NaN,True,1995,swn 80,vwc,pay monthly,unknown,insufficient,shallow well,groundwater,hand pump
86,3688,500.0,Idc,1701,DWE,35.734474,-7.912317,Rufiji,Iringa,Iringa Rural,Magulilwa,441,True,VWC,True,2000,swn 80,vwc,pay monthly,soft,enough,machine dbh,groundwater,hand pump
173,12120,500.0,Dhv,253,RWE,36.703976,-8.120064,Rufiji,Morogoro,Kilombero,Ifakara,450,True,NaN,True,1997,swn 80,vwc,pay monthly,soft,insufficient,shallow well,groundwater,hand pump
192,41034,1000.0,Danida,1712,DWE,35.272504,-8.195502,Rufiji,Iringa,Mufindi,Isalavanu,0,True,VWC,True,0,swn 80,vwc,pay annually,soft,enough,shallow well,groundwater,hand pump
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59142,13850,500.0,Go,337,Go,37.044260,-7.545185,Rufiji,Morogoro,Kilosa,Ruhembe,250,True,VWC,True,1981,swn 80,vwc,pay monthly,soft,enough,shallow well,groundwater,hand pump
59156,23898,300.0,Ir,694,Ir,36.706582,-6.641144,Wami / Ruvu,Morogoro,Kilosa,Kidete,212,True,VWC,True,1997,swn 80,vwc,pay monthly,soft,seasonal,shallow well,groundwater,hand pump
59272,19197,500.0,Co,762,DWE,30.564112,-7.073104,Lake Tanganyika,Rukwa,Nkasi,Kabwe,250,True,NaN,False,2008,swn 80,vwc,pay annually,soft,enough,shallow well,groundwater,hand pump
59316,10273,200.0,Dhv,391,DWE,37.423060,-6.331548,Wami / Ruvu,Morogoro,Mvomero,Mvomero,1,True,VWC,True,1997,swn 80,vwc,pay monthly,soft,insufficient,shallow well,groundwater,hand pump


In [42]:
(X['amount_tsh'] > 1000).sum()

2990

##### Funder

In [43]:
# Shows all categories with 200 wells or more, make an 'Other' column
X['funder'].value_counts().head(44)

Government Of Tanzania            9084
Danida                            3114
Hesawa                            2202
Rwssp                             1374
World Bank                        1349
Kkkt                              1287
World Vision                      1246
Unicef                            1057
Tasaf                              877
District Council                   843
Dhv                                829
Private Individual                 826
Dwsp                               811
0                                  777
Norad                              765
Germany Republi                    610
Tcrs                               602
Ministry Of Water                  590
Water                              583
Dwe                                484
Netherlands                        470
Hifab                              450
Adb                                448
Lga                                442
Amref                              425
Fini Water               

In [44]:
X['funder'].replace(['Rc', 'Roman'], 'Rc Church', inplace=True)
X['funder'].replace('Private Individual', 'Private', inplace=True)
X['funder'].replace(['Finw', 'Fw'], 'Fini Water', inplace=True)
# Jaica and Jica are alternative spellings of Japan International Cooperation Agency
X['funder'].replace('Jaica', 'Jica', inplace=True)
# Kkkt is a church in Tanzania
X['funder'].replace('Kkkt_makwale', 'Kkkt', inplace=True)
X['funder'].replace('Nethalan', 'Netherlands', inplace=True)
X['funder'].replace(['Rwssp', 'Rural Water Supply And Sanitat'], 'Rural Water Supply & Sanitation', inplace=True)
X['funder'].replace('Oxfarm', 'Oxfam', inplace=True)
X['funder'].replace('He', 'Hesawa', inplace=True)
# Norwegian Agency for Development Cooperation
X['funder'].replace('No', 'Norad', inplace=True)
# Spanish acronym for Engineers without borders
X['funder'].replace('Isf', 'Ewb', inplace=True)
# Dwsp is district water and sanitation
# X['funder'].replace('Dwsp', ___)
X['funder'].replace('Mkinga Distric Coun', 'District Council', inplace=True)
X['funder'].replace('Ces (gmbh)', 'Ces(gmbh)').value_counts().head(60)

Government Of Tanzania             9084
Danida                             3114
Hesawa                             2327
Rural Water Supply & Sanitation    1644
Kkkt                               1453
World Bank                         1349
World Vision                       1246
Private                            1121
Unicef                             1057
District Council                    993
Norad                               899
Tasaf                               877
Dhv                                 829
Rc Church                           826
Dwsp                                811
Fini Water                          785
0                                   777
Netherlands                         622
Germany Republi                     610
Tcrs                                602
Ministry Of Water                   590
Water                               583
Jica                                539
Oxfam                               535
Dwe                                 484


##### Installer

In [45]:
# Shows all categories with 200 wells or more, make an 'Other' column
X['installer'].value_counts().head(39)

DWE                           17402
Government                     1825
RWE                            1206
Commu                          1060
DANIDA                         1050
KKKT                            898
Hesawa                          840
0                               777
TCRS                            707
Central government              622
CES                             610
Community                       553
DANID                           552
District Council                551
HESAWA                          539
LGA                             408
World vision                    408
WEDECO                          397
TASAF                           396
District council                392
Gover                           383
AMREF                           329
TWESA                           316
WU                              301
Dmdd                            287
ACRA                            278
World Vision                    270
SEMA                        

##### Local Government Authority

In [46]:
# It may be smart to make a new category that bundles together all the categories
# with less than 200 observations, but probably not :(
X['lga'].value_counts().head(102)

Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Bariadi         1177
Rungwe          1106
                ... 
Kisarawe         223
Temeke           215
Rorya            210
Tarime           209
Ngorongoro       201
Name: lga, Length: 102, dtype: int64

#####  Ward

In [47]:
# way too many different categories, this will probably not be useful outside of imputation
X['ward'].value_counts()

Igosi        307
Imalinyi     252
Siha Kati    232
Mdandu       231
Nduruma      217
            ... 
Mitole         1
Matarawe       1
Chinugulu      1
Ukata          1
Ifinga         1
Name: ward, Length: 2092, dtype: int64

##### Scheme Management

In [48]:
# take a look at the distribution of different categories
X['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [49]:
# Put None, Other, and NaN into one 'Other' category
X['scheme_management'].replace(['None', np.NaN], 'Other', inplace=True)
X['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Other                4644
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
SWC                    97
Trust                  72
Name: scheme_management, dtype: int64

##### Payment

In [50]:
X['payment'].value_counts()

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [51]:
# combine other and unknown
X['payment'].replace('unknown', 'other', inplace=True)
X['payment'].value_counts()

never pay                25348
other                     9211
pay per bucket            8985
pay monthly               8300
pay when scheme fails     3914
pay annually              3642
Name: payment, dtype: int64

##### Management

In [52]:
X['management'].value_counts()

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [53]:
# combine other and unknown
X['management'].replace('unknown', 'other', inplace=True)
X['management'].value_counts()

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
other                1405
water authority       904
company               685
other - school         99
trust                  78
Name: management, dtype: int64

##### Water Quality

In [54]:
X['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [55]:
# combine salty abandoned and flouride abandoned
X['water_quality'].replace(['salty abandoned', 'fluoride abandoned'], 'abandoned', inplace=True)
X['water_quality'].value_counts()

soft         50818
salty         4856
unknown       1876
milky          804
coloured       490
abandoned      356
fluoride       200
Name: water_quality, dtype: int64

##### Source

In [56]:
X['source'].value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [57]:
# combine unknown and other
X['source'].replace('unknown', 'other', inplace=True)
X['source'].value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     278
Name: source, dtype: int64

In [58]:
# source class will be useful for understanding, but shouldn't be used in the models
X['source_class'].value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

In [59]:
# We see that all the information from source will be contained in source_class
X.groupby('source')['source_class'].value_counts()

source                source_class
dam                   surface           656
hand dtw              groundwater       874
lake                  surface           765
machine dbh           groundwater     11075
other                 unknown           278
rainwater harvesting  surface          2295
river                 surface          9612
shallow well          groundwater     16824
spring                groundwater     17021
Name: source_class, dtype: int64

##### Waterpoint Type

#### Observation Removal

In [60]:
# df is still contains 59,400 wells
df = X.merge(y, on='id', how='inner')
df

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type_group,management,payment,water_quality,quantity,source,source_class,waterpoint_type,status_group
0,69572,6000.0,Rc Church,1390,Roman,34.938093,-9.856322,Lake Nyasa,Iringa,Ludewa,Mundindi,109,True,VWC,False,1999,gravity,vwc,pay annually,soft,enough,spring,groundwater,communal standpipe,functional
1,8776,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Mara,Serengeti,Natta,280,NaN,Other,True,2010,gravity,wug,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe,functional
2,34310,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Pangani,Manyara,Simanjiro,Ngorika,250,True,VWC,True,2009,gravity,vwc,pay per bucket,soft,enough,dam,surface,communal standpipe multiple,functional
3,67743,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,Nanyumbu,Nanyumbu,58,True,VWC,True,1986,submersible,vwc,never pay,soft,dry,machine dbh,groundwater,communal standpipe multiple,non functional
4,19728,0.0,Action In A,0,Artisan,31.130847,-1.825359,Lake Victoria,Kagera,Karagwe,Nyakasimbi,0,True,Other,True,0,gravity,other,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,Germany Republi,1210,CES,37.169807,-3.253847,Pangani,Kilimanjaro,Hai,Masama Magharibi,125,True,Water Board,True,1999,gravity,water board,pay per bucket,soft,enough,spring,groundwater,communal standpipe,functional
59396,27263,4700.0,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Rufiji,Iringa,Njombe,Ikondo,56,True,VWC,True,1996,gravity,vwc,pay annually,soft,enough,river,surface,communal standpipe,functional
59397,37057,0.0,NaN,0,NaN,34.017087,-8.750434,Rufiji,Mbeya,Mbarali,Chimala,0,True,VWC,False,0,swn 80,vwc,pay monthly,fluoride,enough,machine dbh,groundwater,hand pump,functional
59398,31282,0.0,Malec,0,Musa,35.861315,-6.378573,Rufiji,Dodoma,Chamwino,Mvumi Makulu,0,True,VWC,True,0,nira/tanira,vwc,never pay,soft,insufficient,shallow well,groundwater,hand pump,functional


In [61]:
# This removes all the observations where all the numerical values are zero
df_zero = df[(df['construction_year'] != 0) | (df['population'] != 0) | (df['amount_tsh'] != 0) | (df['gps_height'] != 0)]
# make a new X and y
X_zero = df_zero.drop('status_group', axis=1)
y_zero = df_zero['status_group']

In [62]:
df_zero.isna().sum()

id                          0
amount_tsh                  0
funder                   1523
gps_height                  0
installer                1528
longitude                   0
latitude                    0
basin                       0
region                      0
lga                         0
ward                        0
population                  0
public_meeting           2355
scheme_management           0
permit                   1950
construction_year           0
extraction_type_group       0
management                  0
payment                     0
water_quality               0
quantity                    0
source                      0
source_class                0
waterpoint_type             0
status_group                0
dtype: int64

### Post-split Cleaning

In [63]:
def get_nums(df):
    return df.select_dtypes(include=['float', 'int'])
GetNumeric = FunctionTransformer(get_nums)

In [64]:
num_subpipe = Pipeline([
    ('scale', RobustScaler())
])

In [65]:
cat_subpipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, drop='first'))
])

In [66]:
#messy_subpipe = Pipeline([
#    ('___', ___)
#])

In [67]:
num_index = []
cat_index = []
messy_index = []
CT = ColumnTransformer([
    ('num', num_subpipe, num_index),
    ('cat', cat_subpipe, cat_index),
#    ('messy', messy_subpipe, messy_index)
])

In [68]:
num_pipe = Pipeline([
    ('num', GetNumeric),
    ('scale', RobustScaler())
])

In [69]:
# train-test split w/ zeroes
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.2)

In [70]:
# trian-test split w/o zeroes
X_zero_train, X_zero_test, y_zero_train, y_zero_test = train_test_split(X_zero, y_zero, random_state=42, test_size=.2)

## Dummy Classifier

In [74]:
dc = DummyClassifier('most_frequent')
dc.fit(X_zero_train_trim, y_zero_train)

DummyClassifier(strategy='most_frequent')

In [75]:
dc.score(X_zero_train_trim, y_zero_train)

0.560767657700173

In [76]:
dc.score(X_zero_test_trim, y_zero_test)

0.5555555555555556

## First Model

In [ ]:
# use SimpleImputer for public_meeting

In [ ]:
# use SimpleImputer for permit as well

### Without Zeroes

In [73]:
X_zero_train.iloc[:, 7].isna().sum()

0

In [74]:
X_zero_train.columns

Index(['id', 'amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'region', 'lga', 'ward', 'population',
       'public_meeting', 'scheme_management', 'permit', 'construction_year',
       'extraction_type_group', 'management', 'payment', 'water_quality',
       'quantity', 'source', 'source_class', 'waterpoint_type'],
      dtype='object')

In [85]:
# reorder the dataframe columns to make it easier to work with
# we want all the numerical columns first and then the categorical columns
num_cols = X_zero_train.select_dtypes(include=['float', 'int', 'int64']).columns
cat_cols = X_zero_train.select_dtypes(exclude=['float', 'int', 'int64']).columns
cols = num_cols.append(cat_cols)

X_zero_train = X_zero_train.reindex(columns=cols)
X_zero_test = X_zero_test.reindex(columns=cols)

In [86]:
# get rid of the unnecessary columns 
X_zero_train_trim = X_zero_train.drop(['id', 'funder', 'installer', 'lga', 'ward', 'source_class'], axis=1)
X_zero_test_trim = X_zero_test.drop(['id', 'funder', 'installer', 'lga', 'ward', 'source_class'], axis=1)


In [173]:
X_zero_train_trim.columns

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population',
       'construction_year', 'basin', 'region', 'public_meeting',
       'scheme_management', 'permit', 'extraction_type_group', 'management',
       'payment', 'water_quality', 'quantity', 'source', 'waterpoint_type'],
      dtype='object')

In [77]:
num_index = list(range(0,6))
cat_index = list(range(6,18))
CT = ColumnTransformer([
    ('num', num_subpipe, num_index),
    ('cat', cat_subpipe, cat_index),
])

In [78]:
dtpipe = Pipeline(steps=[
    ('ct', CT),
    ('tree', DecisionTreeClassifier(max_depth= 10, random_state=42))
])

In [180]:
X_zero_train_trim.index

Int64Index([47905, 31039, 32725, 10913, 30334, 22479, 10323,  3721, 58203,
            25741,
            ...
             2508, 23910, 32875, 55616, 25152,  9351, 16845, 57082,  1286,
            23554],
           dtype='int64', length=31785)

In [90]:
dtpipe.fit(X_zero_train_trim, y_zero_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scale',
                                                                   RobustScaler())]),
                                                  [0, 1, 2, 3, 4, 5]),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  [6, 7, 8, 9, 10, 11, 12, 13,
                                                   14, 15, 16, 17])])),
                ('tree',


In [80]:
cross_val_score(dtpipe, X_zero_train_trim, y_zero_train).mean()

0.766084631115306

In [87]:
dtpipe.score(X_zero_test_trim, y_zero_test)

0.7612935699005914

In [82]:
dtpipe.score(X_zero_train_trim, y_zero_train)

0.7980493943684128

In [95]:
dtpipe.steps[1][1]

DecisionTreeClassifier(max_depth=10, random_state=42)

In [201]:
importances = dtpipe.steps[1][1].feature_importances_

In [108]:
ohe_cols = dtpipe.named_steps['ct'].transformers_[1][1].named_steps['ohe'].get_feature_names()

In [109]:
features = list(num_cols)[1:] + list(ohe_cols)

In [203]:
dict(zip(features, importances))

{'amount_tsh': 0.032234840054863004,
 'gps_height': 0.10982397480524883,
 'longitude': 0.1515654207931356,
 'latitude': 0.14473648616895254,
 'population': 0.07089896344150719,
 'construction_year': 0.08291849635335388,
 'x0_Lake Nyasa': 0.0,
 'x0_Lake Rukwa': 0.00033398110890114284,
 'x0_Lake Tanganyika': 0.000290817162758445,
 'x0_Lake Victoria': 0.0002673727392479518,
 'x0_Pangani': 0.0015680102457061584,
 'x0_Rufiji': 0.0013742393652394923,
 'x0_Ruvuma / Southern Coast': 0.0005196101167785086,
 'x0_Wami / Ruvu': 0.003444891466364545,
 'x1_Dar es Salaam': 0.0002809316497661596,
 'x1_Iringa': 0.016247015837930215,
 'x1_Kigoma': 0.0015067367828325233,
 'x1_Kilimanjaro': 0.000690912167961215,
 'x1_Lindi': 0.001314904269109968,
 'x1_Manyara': 0.0017429666955832179,
 'x1_Mara': 0.0010007054612313328,
 'x1_Morogoro': 0.0017514711128883303,
 'x1_Mtwara': 0.0010347162590799792,
 'x1_Mwanza': 0.0008099614384870081,
 'x1_Pwani': 0.0010528348563485616,
 'x1_Rukwa': 0.0008429759352152592,
 'x1_

## Random Forest Classifier

In [98]:
rfpipe = Pipeline([
    ('ct', CT),
    ('forest', RandomForestClassifier())
])

In [99]:
rfpipe.fit(X_zero_train_trim, y_zero_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scale',
                                                                   RobustScaler())]),
                                                  [0, 1, 2, 3, 4, 5]),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  [6, 7, 8, 9, 10, 11, 12, 13,
                                                   14, 15, 16, 17])])),
                ('forest'

In [100]:
rfpipe.score(X_zero_train_trim, y_zero_train)

0.9999685386188454

In [101]:
rfpipe.score(X_zero_test_trim, y_zero_test)

0.8142695356738392

In [102]:
cross_val_score(rfpipe, X_zero_train_trim, y_zero_train)

array([0.80887211, 0.80840019, 0.81296209, 0.8132767 , 0.80604059])

In [110]:
importances = rfpipe.steps[1][1].feature_importances_

In [111]:
dict(zip(features, importances))

{'amount_tsh': 0.04740818117686883,
 'gps_height': 0.11012134765927781,
 'longitude': 0.1238408805503219,
 'latitude': 0.12485787699898429,
 'population': 0.0763496387301379,
 'construction_year': 0.09074140641881172,
 'x0_Lake Nyasa': 0.0030195069172496243,
 'x0_Lake Rukwa': 0.0014443855322562405,
 'x0_Lake Tanganyika': 0.0037876419740862123,
 'x0_Lake Victoria': 0.0027494453818570626,
 'x0_Pangani': 0.005118137223587138,
 'x0_Rufiji': 0.004148332549891353,
 'x0_Ruvuma / Southern Coast': 0.004441189273061026,
 'x0_Wami / Ruvu': 0.0038097600526100744,
 'x1_Dar es Salaam': 0.0008109180595197596,
 'x1_Iringa': 0.006576848771221855,
 'x1_Kigoma': 0.003484715988859537,
 'x1_Kilimanjaro': 0.003506345921788729,
 'x1_Lindi': 0.001983345781028462,
 'x1_Manyara': 0.002740956315846021,
 'x1_Mara': 0.0023898727511933128,
 'x1_Morogoro': 0.0032544654028099406,
 'x1_Mtwara': 0.00223346469671296,
 'x1_Mwanza': 0.0009118578055299623,
 'x1_Pwani': 0.002745778408702318,
 'x1_Rukwa': 0.00248142543644287

## Gradient Boosting Classifier

In [100]:
gbpipe = Pipeline([
    ('ct', CT),
    ('gb', GradientBoostingClassifier())
])

In [101]:
gbpipe.fit(X_zero_train_trim, y_zero_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scale',
                                                                   RobustScaler())]),
                                                  [0, 1, 2, 3, 6, 10]),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  [4, 5, 7, 8, 9, 11, 12, 13,
                                                   14, 15, 16, 17])])),
                ('gb', Gr

In [104]:
gbpipe.score(X_zero_train_trim, y_zero_train)

0.7771904986628912

In [102]:
gbpipe.score(X_zero_test_trim, y_zero_test)

0.7659494148735372

In [105]:
cross_val_score(gbpipe, X_zero_train_trim, y_zero_train)

array([0.77143307, 0.76655655, 0.77300613, 0.77489382, 0.76577002])

## Random Forest Classifier w/ Grid Search

In [97]:
grid={
    'forest__n_estimators': [50, 100, 200],
    'forest__criterion': ['gini', 'entropy'],
    'forest__max_features': ['auto', 'sqrt']
}

In [112]:
gsrf = GridSearchCV(rfpipe, grid)

In [113]:
gsrf.fit(X_zero_train_trim, y_zero_train)

GridSearchCV(estimator=Pipeline(steps=[('ct',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scale',
                                                                                          RobustScaler())]),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          6,
                                                                          10]),
                                                                        ('cat',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                     

In [115]:
gsrf.score(X_zero_train_trim, y_zero_train)

1.0

In [116]:
gsrf.score(X_zero_test_trim, y_zero_test)

0.8160312067446835

In [117]:
gsrf.best_params_

{'forest__criterion': 'entropy',
 'forest__max_features': 'sqrt',
 'forest__n_estimators': 200}

In [122]:
gsrf.

5

### With Zeroes

In [82]:
# drop unnecessary columns
X_train_trim = X_train.drop(['id', 'funder', 'installer', 'lga', 'ward', 'source_class'], axis=1)
X_test_trim = X_test.drop(['id', 'funder', 'installer', 'lga', 'ward', 'source_class'], axis=1)


In [97]:
dtpipe2 = (Pipeline(steps=[
    ('ct', CT),
    ('tree', DecisionTreeClassifier(random_state=42))
]))

In [98]:
dtpipe2.fit(X_train_trim, y_train)

TypeError: '<' not supported between instances of 'str' and 'int'

In [82]:
cross_val_score(dtpipe2, X_train_trim, y_test)

NameError: name 'dtpipe2' is not defined

In [134]:
dtpipe2.score(X_test_trim, y_test)

0.7551346801346801

# Workbook Ramblings

### Rabbithole Thought on Zeroes

In [42]:
((X['construction_year'] == 0) & (X['population'] == 0)).sum()

20034

In [43]:
((X['construction_year'] == 0) & (X['population'] == 0) & (X['amount_tsh'] == 0)).sum()

19806

In [44]:
X['population'].value_counts()

0       21381
1        7025
200      1940
150      1892
250      1681
        ...  
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64

In [45]:
((X['construction_year'] == 0) & (X['population'] == 0) & (X['amount_tsh'] == 0) & (X['gps_height'] == 0)).sum()

19668

In [46]:
nonfunct = df['status_group'] == 'non functional'

In [47]:
((X['construction_year'] == 0) & (X['population'] == 0) & (X['amount_tsh'] == 0) & (X['gps_height'] == 0) & nonfunct).sum()

7907

In [48]:
functional = ((df['status_group'] == 'functional') | (df['status_group'] == 'functional needs repair'))

In [49]:
((X['construction_year'] == 0) & (X['population'] == 0) & (X['amount_tsh'] == 0) & (X['gps_height'] == 0) & functional).sum()

11761

In [50]:
11761 / 7907

1.487416213481725

In [51]:
functional.sum() / nonfunct.sum()

1.6025236593059937

### Constuction Year

In [52]:
(X['construction_year'] == 0).sum()

20709

In [53]:
# nothing below 1960, possibly bin the construction by decade
X['construction_year'].value_counts().

Int64Index([   0, 2010, 2008, 2009, 2000, 2007, 2006, 2003, 2011, 2004, 2012,
            2002, 1978, 1995, 2005, 1999, 1998, 1990, 1985, 1980, 1996, 1984,
            1982, 1994, 1972, 1974, 1997, 1992, 1993, 2001, 1988, 1983, 1975,
            1986, 1976, 1970, 1991, 1989, 1987, 1981, 1977, 1979, 1973, 2013,
            1971, 1960, 1967, 1963, 1968, 1969, 1964, 1962, 1961, 1965, 1966],
           dtype='int64')

In [78]:
X[X['construction_year'] == 0].head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
4,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
6,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365,Kwa Ngomho,0,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
7,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
8,0.0,2012-11-03,Wateraid,0,Water Aid,32.711100,-5.146712,Kwa Ramadhan Musa,0,Lake Tanganyika,Imalauduki,Tabora,14,6,Tabora Urban,Itetemia,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump
9,0.0,2011-08-03,Isingiro Ho,0,Artisan,30.626991,-1.257051,Kwapeto,0,Lake Victoria,Mkonomre,Kagera,18,1,Karagwe,Kaisho,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


### Population cleaning

In [54]:
(X['population'] == 0).sum()

21381

### Total Static Head Cleaning

In [55]:
# almost two thirds of the data is 0, unlikely
(X['amount_tsh'] == 0).sum()

41639

In [56]:
# if it is surface level, it makes sense for the tsh to be zero
((X['source_class'] == 'surface') & (X['amount_tsh'] == 0)).sum()

7964

In [57]:
# but sometimes, even surface water doesn't have a tsh of zero
((X['source_class'] == 'surface') & (X['amount_tsh'] != 0)).sum()

5364

In [58]:
functional = ((df['status_group'] == 'functional') | (df['status_group'] == 'functional needs repair'))

In [59]:
nonfunct = df['status_group'] == 'non functional'

In [60]:
ztsh = df['amount_tsh'] == 0

In [61]:
# while many of these probably don't make sense
df[functional & ztsh].shape

(22754, 41)

In [62]:
# these definitely make sense
df[nonfunct & ztsh].shape

(18885, 41)

In [63]:

X.groupby('extraction_type_group')['amount_tsh'].mean()

extraction_type_group
afridev            330.353107
gravity            411.739186
india mark ii      528.016667
india mark iii      14.438776
mono               322.514834
nira/tanira         71.746382
other               82.788367
other handpump     579.612637
other motorpump    491.803279
rope pump          203.392461
submersible        398.341034
swn 80             304.760218
wind-powered        40.982906
Name: amount_tsh, dtype: float64

In [64]:
nosh = X[X['amount_tsh'] == 0]
nosh['region'].value_counts()

Shinyanga        4967
Mbeya            4639
Kagera           3316
Mwanza           3059
Arusha           2435
Kilimanjaro      2354
Dodoma           2201
Morogoro         2041
Tabora           1959
Iringa           1852
Tanga            1661
Mara             1602
Kigoma           1319
Singida          1265
Pwani            1220
Ruvuma           1208
Rukwa            1141
Lindi            1084
Mtwara           1036
Manyara          1010
Dar es Salaam     270
Name: region, dtype: int64

In [65]:
X['region'].value_counts()

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64

In [66]:
nosh['region'].value_counts()[:]/X['region'].value_counts()[:]

Arusha           0.726866
Dar es Salaam    0.335404
Dodoma           1.000000
Iringa           0.349830
Kagera           1.000000
Kigoma           0.468395
Kilimanjaro      0.537566
Lindi            0.701164
Manyara          0.638029
Mara             0.813611
Mbeya            1.000000
Morogoro         0.509486
Mtwara           0.598844
Mwanza           0.986138
Pwani            0.462998
Rukwa            0.631084
Ruvuma           0.457576
Shinyanga        0.996989
Singida          0.604396
Tabora           1.000000
Tanga            0.652140
Name: region, dtype: float64

### Elevation

In [67]:
X.groupby('basin')['gps_height'].mean()

basin
Internal                    885.677328
Lake Nyasa                  796.058407
Lake Rukwa                  583.585575
Lake Tanganyika             715.002954
Lake Victoria               328.424961
Pangani                    1034.890045
Rufiji                      880.665873
Ruvuma / Southern Coast     410.640329
Wami / Ruvu                 206.458827
Name: gps_height, dtype: float64

In [68]:
X.corr()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
amount_tsh,1.000000,0.076650,0.022134,-0.052670,0.002944,-0.026813,-0.023599,0.016288,0.067915
gps_height,0.076650,1.000000,0.149155,-0.035751,0.007237,-0.183521,-0.171233,0.135003,0.658727
longitude,0.022134,0.149155,1.000000,-0.425802,0.023873,0.034197,0.151398,0.086590,0.396732
latitude,-0.052670,-0.035751,-0.425802,1.000000,0.006837,-0.221018,-0.201020,-0.022152,-0.245278
num_private,0.002944,0.007237,0.023873,0.006837,1.000000,-0.020377,-0.004478,0.003818,0.026056
region_code,-0.026813,-0.183521,0.034197,-0.221018,-0.020377,1.000000,0.678602,0.094088,0.031724
district_code,-0.023599,-0.171233,0.151398,-0.201020,-0.004478,0.678602,1.000000,0.061831,0.048315
population,0.016288,0.135003,0.086590,-0.022152,0.003818,0.094088,0.061831,1.000000,0.260910
construction_year,0.067915,0.658727,0.396732,-0.245278,0.026056,0.031724,0.048315,0.260910,1.000000


In [69]:
X[X['gps_height'] == 0]['construction_year'].value_counts()

0       19675
1984       91
2008       62
1995       56
1971       46
2009       45
2007       43
2005       42
2006       33
2010       32
1980       23
1997       20
1992       18
2001       17
1974       17
2004       17
2011       15
2002       14
1989       13
1975       13
1994       13
1990       12
2000       12
1970       11
1985       10
1979       10
2003        9
1999        9
1981        9
1998        8
1982        8
1986        5
1993        4
1996        3
1977        3
1988        3
1978        3
1987        3
1983        2
2012        1
1961        1
1965        1
1972        1
1966        1
1962        1
1973        1
1968        1
1976        1
Name: construction_year, dtype: int64

In [70]:
X['construction_year'].value_counts()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [71]:
df[df['gps_height'] ==0]['date_recorded'].dt.month.value_counts()

7     5690
10    4319
4     2926
3     2478
8     2242
2      978
1      866
11     744
12     170
9       24
5        1
Name: date_recorded, dtype: int64

In [72]:
df['date_recorded'].dt.year.value_counts()

2011    28674
2013    24271
2012     6424
2004       30
2002        1
Name: date_recorded, dtype: int64

In [73]:
notzero = X[X['gps_height'] != 0]

In [74]:
notzero.groupby('ward')['gps_height'].mean()

ward
Aghondi          1314.333333
Akheri           1369.955556
Arash            1891.250000
Arri             1585.315789
Arusha Chini      725.228070
                    ...     
Yombo               8.913462
Zinga/Ikerege      -4.647482
Zirai             162.285714
Ziwani            -10.709677
Zombo             490.080000
Name: gps_height, Length: 1316, dtype: float64

In [75]:
notzero.groupby('lga')['gps_height'].mean()

lga
Arusha Rural    1429.448083
Arusha Urban    1349.174603
Babati          1401.093933
Bagamoyo         118.901822
Bariadi         1350.981707
                   ...     
Tarime          1514.712919
Temeke           -11.630841
Tunduru          553.432624
Ukerewe         1197.322581
Ulanga           376.296241
Name: gps_height, Length: 83, dtype: float64

In [76]:
notzero.groupby('region')['gps_height'].mean()

region
Arusha           1401.144179
Dar es Salaam      31.221945
Iringa           1697.442010
Kigoma           1235.562145
Kilimanjaro      1180.891528
Lindi             213.418288
Manyara          1426.077701
Mara             1341.625190
Morogoro          405.250665
Mtwara            258.410301
Mwanza           1199.643052
Pwani              83.335504
Rukwa            1404.461283
Ruvuma            903.740530
Shinyanga        1350.981707
Singida          1416.857621
Tanga             667.914800
Name: gps_height, dtype: float64